In [1]:
!pip install pytorchvideo
!pip install torchsummary
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 615.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 20.3 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=dcc2487229807134bbe5c5add08bb6ebf3bf674f3f1c89187b65dfd63f43993a
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=93a1d3bdb0d1d3142529e0406cbd549e239d732f97e5345e301c27996bf69bcd
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pytorchvideo fvcore


In [2]:
import torch
from imblearn.over_sampling import SMOTE
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize
#from torchvision.transforms._transforms_video import CenterCropVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models

import sys
from torchsummary import summary






class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [3]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60







def transform_data(x):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [
                Lambda(lambda x: x/255.0),
                Normalize(([0.2933, 0.4799, 0.6723]), ([0.1576, 0.1674, 0.2477])),
                Grayscale(num_output_channels=1),
                CenterCrop([720,900]),
                Resize([512,512]),
                Lambda(lambda x: x.permute(1,2,3,0)),#(channel, frames(depth), height, width)
#                 Lambda(lambda x: print(x.shape))

            ]

        ),
    )
    
    return transform(x)

def lp_video(video,start_time, end_time):
    global means,stds
    global nst
    
    video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)
 
    if video_data["video"] is None:
        return None        # or pass
    else:
    
        video_data["video"] = video_data["video"].permute(1,0,2,3)
        
        video_data = transform_data( video_data)

        return video_data["video"]

# def load_dataloaders(batch_size,start,end):

    
#     train_inputs =[] #x
#     train_classes = [] #y
#     global neg

#     with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
#         data = pickle.load(f)


# # keys are files so iterate only limited files due to memory limitations.
#     for key in list(data.keys())[start:end]:
        
#         filename = key
#         
#         video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
#     # file functions
        
#         for x in data[key]:
#             if x[0] in range(0,5):
#                 img_cls = x[0]



#                 # start_time = x[1]
#                 # end_time = x[2]
#                 vid = lp_video(video,x[1], x[2])

#                 for m in torch.unbind(vid, dim=3):
                    
#                     train_classes.append(img_cls)
#                     train_inputs.append(m.view(-1).numpy())
                
                
                


#  #weights =torch.tensor([166.0, 370.0, 63.0, 232.0, 905.0, 102.0, 537.0, 993.0, 133.0, 9:154.0, 286.0, 435.0, 452.0, 779.0, 354.0, 136.0, 232.0, 2154.0, 374.0, 787.0, 337.0, 268.0, 954.0, 167.0, 543.0, 324.0, 669.0, 333.0, 211.0, 433.0, 140.0, 665.0, 107.0, 305.0, 162.0, 61.0, 1229.0, 958.0, 897.0, 543.0, 625.0, 334.0, 776.0, 111.0, 412.0, 583.0, 255.0, 103.0, 108.0, 201.0, 384.0, 632.0, 13.0, 322.0, 664.0, 435.0, 194.0, 322.0, 113.0, 462.0,2330.0])

# #if under 100 ignore
# #count round of train
# #     sampling_stat = {0:3}
#     sampling_stat = {0:500,1:500,2:70,3:500}  
#     over = SMOTE(sampling_strategy=sampling_stat,k_neighbors=60)
#     x_over, y_over = over.fit_resample(np.array(train_inputs), np.array(train_classes))
#     print("Class:{} Count:{}".format(*np.unique(y_over, return_counts=True)))  
 
#     x_over = torch.from_numpy(x_over).view(-1,1,512,512)
    
   
#     print(x_over[0].shape)
#     print(x_over.shape)
  
            
#     signds = SignDataset(x_over, y_over)
#     trainlen = int(len(signds)*0.8)
#     torch.manual_seed(0)
    
#     train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
#     trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

#     valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
#     return trainloader, valloader
        
    

In [4]:
def load_testloader(batch_size,start,end):

    time_start = []
    time_end = []
    train_inputs =[] #x
    train_classes = [] #y
    

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


       

            
        
            

    signds = SignDataset(train_inputs, train_classes)
    
    

    testloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [5]:
def load_vid(batch_size,start,end):


    train_inputs =[] #x

    video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Val_Set/VALIDATION/MSSL_VAL_SET_VIDEOS/p01_n131.mp4")
    # file functions

    vid = lp_video(video,start, end)

    for m in torch.unbind(vid, dim=3):
        train_inputs.append(m)

    

    testloader = DataLoader(train_inputs, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [6]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""



__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=60, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(1, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

In [7]:

def evluate(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, data in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            print(pred)



In [8]:
def test(model, device, loader,criterion, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    test_loss = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            test_loss += criterion(output, target)
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            #print("correct", correct)
    test_loss /= num_samples
    print(' Val Accuracy: {}/{} ({:.0f}%) Val Loss: {:.6f}'.format(correct, num_samples,100. * correct / num_samples, test_loss))
    return  test_loss

In [9]:
def train(model, device, train_loader,validloader,scheduler, optimizer,criterion,epochs,lr):
    print("Train start")
    breakout = False
    
    last_loss = 100
    patience = 3
    triggertimes = 0
    
    
    model.half()
    model.cuda()

    
    

    optimizer.param_groups[0]['lr'] = lr
    
    for epoch in range(epochs):
        model.train()
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
        if optimizer.param_groups[0]['lr'] < 1e-5:    
            scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 93:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            current_loss = test(model,device,validloader,criterion, mode = "Validating")
            
        
            print("breakout")
            return model, optimizer
            
            

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        
        current_loss = test(model,device,validloader,criterion)
        if current_loss > last_loss:
            trigger_times += 1
            print('Trigger Times for early stopping:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model, optimizer

        else:
            print('trigger times: 0')
            trigger_times = 0

        last_loss = current_loss
    
    
        
    return model, optimizer

In [10]:
n_classes = 60


model = nn.Sequential(OrderedDict([
    ('frontend',effnetv2_m())
]))





# specify loss function (categorical cross-entropy)

# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        
        {"params": model.frontend.parameters(), "lr": 1e-2},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



criterion = nn.CrossEntropyLoss()

state = torch.load("../input/fmosel/model_optimizer312.pt")
model.load_state_dict(state['model_state_dict'])
model.half()
model.cuda()
optimizer.load_state_dict(state['optimizer_state_dict'])

In [11]:
#print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [12]:
#summary(model, (1, 512, 640), device='cpu')

In [13]:


def load_dataloaders(batch_size,start,end):

    
    train_inputs =[] #x
    train_classes = [] #y
    global neg

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        
        filename = key
        
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions
        
        for x in data[key]:
            if x[0] in range(50,53):
                img_cls = x[0]



                # start_time = x[1]
                # end_time = x[2]
                vid = lp_video(video,x[1], x[2])

                for m in torch.unbind(vid, dim=3):
                    
                    train_classes.append(img_cls)
                    train_inputs.append(m.view(-1).numpy())
                
                
                


 #50:384.0, 632.0, 13.0, 53:322.0, 664.0, 

#if under 100 ignore
#count round of train
#     sampling_stat = {0:3}
    sampling_stat = {50:500}  
    over = SMOTE(sampling_strategy=sampling_stat,k_neighbors=8)
    x_over, y_over = over.fit_resample(np.array(train_inputs), np.array(train_classes))
    print("Class:{} Count:{}".format(*np.unique(y_over, return_counts=True)))  
 
    x_over = torch.from_numpy(x_over).view(-1,1,512,512)
    
   
    print(x_over[0].shape)
    print(x_over.shape)
  
            
    signds = SignDataset(x_over, y_over)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    

In [14]:
trainloader, valloader = load_dataloaders(batch_size=16,start=0, end=112)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

Class:[50 51 52] Count:[500 634  13]
torch.Size([1, 512, 512])
torch.Size([1147, 1, 512, 512])
Train start
Epoch: 1 , Training Accuracy: 113/917 (12%) Training Loss: 0.264269
 Val Accuracy: 127/230 (55%) Val Loss: 0.092590
trigger times: 0
Epoch: 2 , Training Accuracy: 783/917 (85%) Training Loss: 0.036950
 Val Accuracy: 219/230 (95%) Val Loss: 0.014915
trigger times: 0
Epoch: 3 , Training Accuracy: 893/917 (97%) Training Loss: 0.008905
 Val Accuracy: 223/230 (97%) Val Loss: 0.009163
breakout


In [15]:



def load_dataloaders(batch_size,start,end):

    
    train_inputs =[] #x
    train_classes = [] #y
    global neg

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        
        filename = key
        
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions
        
        for x in data[key]:
            if x[0] in range(55,58):
                img_cls = x[0]



                # start_time = x[1]
                # end_time = x[2]
                vid = lp_video(video,x[1], x[2])

                for m in torch.unbind(vid, dim=3):
                    
                    train_classes.append(img_cls)
                    train_inputs.append(m.view(-1).numpy())
                
                
                


 #435.0, 194.0, ,57:322.0, 113.0, 462.0

#if under 100 ignore
#count round of train
#     sampling_stat = {0:3}
    sampling_stat = {55:500,56:500, 57:500}  
    over = SMOTE(sampling_strategy=sampling_stat,k_neighbors=60)
    x_over, y_over = over.fit_resample(np.array(train_inputs), np.array(train_classes))
    print("Class:{} Count:{}".format(*np.unique(y_over, return_counts=True)))  
 
    x_over = torch.from_numpy(x_over).view(-1,1,512,512)
    
   
    print(x_over[0].shape)
    print(x_over.shape)
  
            
    signds = SignDataset(x_over, y_over)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    

In [16]:
trainloader, valloader = load_dataloaders(batch_size=16,start=0, end=112)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (500) in class 55 will be larger than the number of samples in the majority class (class #55 -> 435)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (500) in class 56 will be larger than the number of samples in the majority class (class #55 -> 435)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (500) in class 57 will be larger than the number of samples in the majority class (class #55 -> 435)
  f"After over-sampling, the number of samples ({n_samples})"


Class:[55 56 57] Count:[500 500 500]
torch.Size([1, 512, 512])
torch.Size([1500, 1, 512, 512])
Train start
Epoch: 1 , Training Accuracy: 91/1200 (8%) Training Loss: 0.265535
 Val Accuracy: 185/300 (62%) Val Loss: 0.119385
trigger times: 0
Epoch: 2 , Training Accuracy: 1017/1200 (85%) Training Loss: 0.047064
 Val Accuracy: 218/300 (73%) Val Loss: 0.042542
trigger times: 0
Epoch: 3 , Training Accuracy: 1181/1200 (98%) Training Loss: 0.006174
 Val Accuracy: 295/300 (98%) Val Loss: 0.005943
breakout


In [17]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer313.pt")

In [18]:



def load_dataloaders(batch_size,start,end):

    
    train_inputs =[] #x
    train_classes = [] #y
    global neg

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        
        filename = key
        
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions
        
        for x in data[key]:
            if x[0] == 17:
                img_cls = x[0]



                # start_time = x[1]
                # end_time = x[2]
                vid = lp_video(video,x[1], x[2])

                for m in torch.unbind(vid, dim=3):
                    
                    train_classes.append(img_cls)
                    train_inputs.append(m.view(-1).numpy())
    
    split = len(train_inputs)//3
    train_inputs = train_inputs[:split]
    train_classes = train_classes[:split]
    for key in list(data.keys())[start:end]:
        
        filename = key
        
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions
        
        for x in data[key]:
            if x[0] in [59,30]:
                img_cls = x[0]



                # start_time = x[1]
                # end_time = x[2]
                vid = lp_video(video,x[1], x[2])

                for m in torch.unbind(vid, dim=3):
                    
                    train_classes.append(img_cls)
                    train_inputs.append(m.view(-1).numpy())
                
                
                


 #435.0, 194.0, ,57:322.0, 113.0, 462.0

#if under 100 ignore
#count round of train
#     sampling_stat = {0:3}
    sampling_stat = {59:500,30:500}  
    over = SMOTE(sampling_strategy=sampling_stat,k_neighbors=60)
    x_over, y_over = over.fit_resample(np.array(train_inputs), np.array(train_classes))
    print("Class:{} Count:{}".format(*np.unique(y_over, return_counts=True)))  
 
    x_over = torch.from_numpy(x_over).view(-1,1,512,512)
    
   
    print(x_over[0].shape)
    print(x_over.shape)
  
            
    signds = SignDataset(x_over, y_over)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    

In [19]:
trainloader, valloader = load_dataloaders(batch_size=16,start=0, end=112)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

Class:[17 30 59] Count:[718 500 500]
torch.Size([1, 512, 512])
torch.Size([1718, 1, 512, 512])
Train start
Epoch: 1 , Training Accuracy: 217/1374 (16%) Training Loss: 0.247932
 Val Accuracy: 226/344 (66%) Val Loss: 0.091553
trigger times: 0
Epoch: 2 , Training Accuracy: 1179/1374 (86%) Training Loss: 0.032412
 Val Accuracy: 336/344 (98%) Val Loss: 0.010422
trigger times: 0
Epoch: 3 , Training Accuracy: 1360/1374 (99%) Training Loss: 0.005246
 Val Accuracy: 338/344 (98%) Val Loss: 0.004166
breakout


In [20]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer315.pt")

In [21]:
#not done on 16:500,41:500,21:500,25:500 ,29 ,34:500,45, ,49:500,,53:500,54,, 58:500,44:500 also